In [4]:
import pickle as pkl
from gensim.models import Word2Vec
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

DATA_DIR = '../data/'

In [5]:
# load data from saved files - preprocessed data for notes and diagnosis codes

df_diag_diabetes_hadm_ids = pkl.load(open(f'{DATA_DIR}diag_diabetes_hadm_ids.p','rb'))
print('df_diag_diabetes_hadm_ids.shape:   ', df_diag_diabetes_hadm_ids.shape)

df_diag_icd9 = pkl.load(open(f'{DATA_DIR}diag_icd9.p','rb'))
print('df_diag_icd9.shape:                ', df_diag_icd9.shape)

df_diag_icd9_rolled = pkl.load(open(f'{DATA_DIR}diag_icd9_rolled.p','rb'))
print('df_diag_icd9_rolled.shape:         ', df_diag_icd9_rolled.shape)

df_notes = pkl.load(open(f'{DATA_DIR}notes_final.p','rb'))
print('df_notes.shape:                    ', df_notes.shape)

notes_tokens_list = pkl.load(open(f'{DATA_DIR}notes_tokens_list.p','rb'))
print('notes_tokens_list.length:          ', len(notes_tokens_list))

icd9_unique_list = pkl.load(open(f'{DATA_DIR}diag_icd9_unique_list.p','rb'))
print('icd9_unique_list.len:              ', len(icd9_unique_list))

icd9_rolled_unique_list = pkl.load(open(f'{DATA_DIR}diag_icd9_rolled_unique_list.p','rb'))
print('icd9_rolled_unique_list.len:       ', len(icd9_rolled_unique_list))

df_diag_diabetes_hadm_ids.shape:    (14222,)
df_diag_icd9.shape:                 (14222, 3)
df_diag_icd9_rolled.shape:          (14222, 3)
df_notes.shape:                     (399631, 8)
notes_tokens_list.length:           53229


In [9]:
# Statistics for comparison with original paper statistics after preprocessing

print('Num. of used records             ', df_notes.shape[0])
print('Num. of regular labels           ',len(icd9_unique_list))
print('Num. of rolled up labels         ',len(icd9_rolled_unique_list))
print('Num. of unique tokens            ',len(notes_tokens_list))
print('Avg. num. of tokens per report   ',df_notes['NTOKENS_LEN'].sum() / len(df_notes['NTOKENS_LEN']))

Num. of used records              399631
Num. of regular labels            4103
Num. of rolled up labels          781
Num. of unique tokens             53229
Avg. num. of tokens per report    309.09424193818796


In [14]:
# Train word2vec model, commented as model is already stored to file
model = Word2Vec(df_notes['NTOKENS'].to_list(), min_count=1, vector_size=300, workers=4, sg=1)
model.save('../data/word2vec_model.model')

In [19]:
model = Word2Vec.load('../data/word2vec_model.model')
print(model)

Word2Vec(vocab=53278, vector_size=300, alpha=0.025)


In [22]:
x = torch.empty((len(df_notes['NTOKENS'])))
print(type(x))

<class 'torch.Tensor'>


In [57]:
for idx, notes in enumerate(df_notes['NTOKENS']):
  if idx < 0:
    continue
  if idx % 1000 == 0:
   print(idx)
  t1 = model.wv[df_notes['NTOKENS'][idx]]
  if idx == 0:
    x = torch.from_numpy(t1).unsqueeze(0)
  else:
    t2 = torch.from_numpy(t1).unsqueeze(0)
    print(x.shape, t2.shape)
    x = pad_sequence([x, t2])
    print(x.shape)

  if idx == 10000:
    break

0
torch.Size([1, 428, 300]) torch.Size([1, 306, 300])


RuntimeError: The size of tensor a (428) must match the size of tensor b (306) at non-singleton dimension 1